In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import skimage
from skimage import io, feature, measure, transform
import tensorflow as tf
from pandas import Series,DataFrame
import sampling as smp

In [2]:
def reduce(image, scale=4):
    reducedimage = transform.pyramid_reduce(image, downscale=scale)
    return reducedimage

In [3]:
training = pd.read_msgpack('training.bin')
non_empty_training = training[~training.maskC.isnull()]
non_empty_training.index=np.arange(len(non_empty_training))
centers = non_empty_training.maskC.as_matrix()
center_list=np.array([tuple(c) for c in centers])

In [4]:
training_centers=center_list[0:1000]

In [5]:
#### crop image to 80 x 136 to make the numbers rounder for the nn..
def build_training_data(num_images):
    data=[]
    for i in range(num_images):
        if i%100 ==0:
            print 'Reading patch ', i
        ultra_image = smp.image_pair(non_empty_training.ix[i].subject,non_empty_training.ix[i].img).image
        data.append(reduce(ultra_image)[25:,:136].flatten())
    return np.array(data)

In [6]:
time X=build_training_data(1000)

Reading patch  0
Reading patch  100
Reading patch  200
Reading patch  300
Reading patch  400
Reading patch  500
Reading patch  600
Reading patch  700
Reading patch  800
Reading patch  900
CPU times: user 1min 4s, sys: 5.01 s, total: 1min 9s
Wall time: 1min 17s


In [7]:
sess=tf.InteractiveSession()

### use 4x reduced images as input features
x= tf.placeholder(tf.float32,shape=[None, 10880])
x_image = tf.reshape(x, [-1,80,136,1])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

In [8]:
X[0].shape

(10880,)

In [9]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [10]:
### helper functions to build Conv and MaxPool layers

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x,psize):
  return tf.nn.max_pool(x, ksize=[1, psize, psize, 1],
                        strides=[1, psize, psize, 1], padding='SAME')

In [11]:
### weights and biases for the first Conv layer; 5x5 filter and 32 feat.
W_conv1 = weight_variable([7, 7, 1, 32])
b_conv1 = bias_variable([32])

In [12]:
### build first Conv and MaxPool layers
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1,2)

In [13]:
h_pool1.get_shape()

TensorShape([Dimension(None), Dimension(40), Dimension(68), Dimension(32)])

In [14]:
### stack a second Conv + MaxPool layer; again with 32 feat.
W_conv2 = weight_variable([5, 5, 32, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2,2)


In [15]:
h_pool2.get_shape()

TensorShape([Dimension(None), Dimension(20), Dimension(34), Dimension(32)])

In [16]:
### stack a third Conv + MaxPool layer; this time 64 feat.
W_conv3 = weight_variable([5, 5, 32, 64])
b_conv3 = bias_variable([64])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool(h_conv3,2)


In [17]:
h_pool3.get_shape()

TensorShape([Dimension(None), Dimension(10), Dimension(17), Dimension(64)])

In [18]:
#### add fully-connected layer

W_fc1 = weight_variable([10 * 17 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, 10*17*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [19]:
### dropout

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [20]:
### output layer!
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv=(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [21]:
### use least squares cost function
cost_fn = tf.reduce_mean(tf.square(y_conv - y_))

In [22]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost_fn)

In [2]:
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

NameError: name 'sess' is not defined

In [24]:
accuracy =  tf.reduce_mean(tf.square(y_conv - y_))

In [25]:
for i in range(10000):
    pts = np.random.randint(0,1000,size=50)
    batch = (X[pts].flatten().reshape(50,10880), training_centers[pts].flatten().reshape(50,2))
    if i%10 ==0:
        train_error = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
        print 'RMS error on training batch',i, ':',  train_error
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob: 0.5})


RMS error on training batch 0 : 70134.3
RMS error on training batch 10 : 2706.78
RMS error on training batch 20 : 2768.83
RMS error on training batch 30 : 2814.41
RMS error on training batch 40 : 2676.06
RMS error on training batch 50 : 3271.0
RMS error on training batch 60 : 2338.64
RMS error on training batch 70 : 1752.29
RMS error on training batch 80 : 1883.41
RMS error on training batch 90 : 1582.06
RMS error on training batch 100 : 1183.68
RMS error on training batch 110 : 1355.92
RMS error on training batch 120 : 1368.09
RMS error on training batch 130 : 1496.86
RMS error on training batch 140 : 1900.79
RMS error on training batch 150 : 1291.38
RMS error on training batch 160 : 1497.03
RMS error on training batch 170 : 2045.68
RMS error on training batch 180 : 972.466
RMS error on training batch 190 : 1489.89
RMS error on training batch 200 : 821.175
RMS error on training batch 210 : 1371.78
RMS error on training batch 220 : 1250.38
RMS error on training batch 230 : 853.41
RMS e

KeyboardInterrupt: 

In [32]:
def build_test_data(num_images):
    data=[]
    for i in range(1000,num_images+1000):
        if i%100 ==0:
            print 'Reading patch ', i
        ultra_image = smp.image_pair(non_empty_training.ix[i].subject,non_empty_training.ix[i].img).image
        data.append(reduce(ultra_image)[25:,:136].flatten())
    return np.array(data)

In [3]:
saved_model  = saver.save(sess, "/Users/gus/CDIPS/nerve-project/")

NameError: name 'saver' is not defined

In [ ]:
time x_test=build_test_data(1000)

Reading patch  1000
Reading patch  1100


In [35]:
x_test.shape

(1000, 10880)

In [36]:
test_centers=center_list[1000:2000]

In [1]:
test_centers.shape

NameError: name 'test_centers' is not defined

In [ ]:
print("test rms error %g"%accuracy.eval(feed_dict={
    x: x_test, y_: test_centers, keep_prob: 1.0}))